 ```{contents}
```

## Intiution

### 1. Theoretical Intuition of XGBoost

At its core, **XGBoost = Gradient Boosting + Regularization + System Optimizations**.

* It builds trees sequentially, where each new tree corrects the **residual errors** of the previous ensemble.
* It optimizes a **custom loss function** using **gradient descent in function space**.
* Unlike plain Gradient Boosting, XGBoost adds **regularization**, **second-order optimization**, and **clever system-level tricks** (like sparsity awareness, cache optimization, parallelism).

So, XGBoost is like a **regularized, optimized version of Gradient Boosting**.

---

### 2. Mathematical Intuition

Suppose we have a dataset:

$$
D = \{(x_i, y_i)\}, \quad i=1,2,\dots,n
$$

where $x_i \in \mathbb{R}^d$, and $y_i$ is the target.

Our goal: learn a function $\hat{y}_i = \sum_{m=1}^M f_m(x_i)$ where each $f_m$ is a regression tree.

---

#### Step 1: Define Objective Function

The general objective is:

$$
\text{Obj} = \sum_{i=1}^n l(y_i, \hat{y}_i) + \sum_{m=1}^M \Omega(f_m)
$$

Where:

* $l(y_i, \hat{y}_i)$ → differentiable loss function (MSE, log-loss, etc.)
* $\Omega(f)$ → regularization to control complexity:

  $$
  \Omega(f) = \gamma T + \frac{1}{2} \lambda \sum_{j=1}^T w_j^2
  $$

  * $T$ = number of leaves in the tree
  * $w_j$ = score (weight) of leaf $j$
  * $\gamma, \lambda$ = regularization parameters

---

#### Step 2: Boosting Iteration

Suppose after $t-1$ iterations, the model is:

$$
\hat{y}_i^{(t-1)} = \sum_{k=1}^{t-1} f_k(x_i)
$$

Now we add a new function $f_t(x)$:

$$
\hat{y}_i^{(t)} = \hat{y}_i^{(t-1)} + f_t(x_i)
$$

The objective becomes:

$$
\text{Obj}^{(t)} = \sum_{i=1}^n l\big(y_i, \hat{y}_i^{(t-1)} + f_t(x_i)\big) + \Omega(f_t)
$$

---

#### Step 3: Taylor Expansion (2nd Order Approximation)

Since $l$ can be complex, XGBoost uses **second-order Taylor expansion**:

$$
l(y_i, \hat{y}_i^{(t-1)} + f_t(x_i)) \approx l(y_i, \hat{y}_i^{(t-1)}) + g_i f_t(x_i) + \tfrac{1}{2} h_i f_t(x_i)^2
$$

Where:

* $g_i = \frac{\partial l(y_i, \hat{y}_i^{(t-1)})}{\partial \hat{y}_i^{(t-1)}}$ (first derivative → gradient)
* $h_i = \frac{\partial^2 l(y_i, \hat{y}_i^{(t-1)})}{\partial (\hat{y}_i^{(t-1)})^2}$ (second derivative → curvature/Hessian)

So, new objective becomes:

$$
\text{Obj}^{(t)} \approx \sum_{i=1}^n \Big[ g_i f_t(x_i) + \tfrac{1}{2} h_i f_t(x_i)^2 \Big] + \Omega(f_t)
$$

---

#### Step 4: Tree Structure Optimization

For tree $f_t(x)$, prediction for leaf $j$ is constant $w_j$.
If sample $i$ belongs to leaf $j$, then $f_t(x_i) = w_j$.

So:

$$
\text{Obj}^{(t)} = \sum_{j=1}^T \left[ G_j w_j + \tfrac{1}{2}(H_j + \lambda) w_j^2 \right] + \gamma T
$$

Where:

* $G_j = \sum_{i \in I_j} g_i$ (sum of gradients in leaf $j$)
* $H_j = \sum_{i \in I_j} h_i$ (sum of Hessians in leaf $j$)
* $I_j$ = set of data points in leaf $j$

---

#### Step 5: Optimal Leaf Weight

To minimize objective, take derivative wrt $w_j$:

$$
w_j^* = -\frac{G_j}{H_j + \lambda}
$$

---

#### Step 6: Optimal Split (Gain)

The score of a tree:

$$
\text{Obj}^{(t)} = -\tfrac{1}{2} \sum_{j=1}^T \frac{G_j^2}{H_j + \lambda} + \gamma T
$$

When splitting a node into left/right child, the **Gain** is:

$$
\text{Gain} = \tfrac{1}{2} \left[ \frac{G_L^2}{H_L + \lambda} + \frac{G_R^2}{H_R + \lambda} - \frac{(G_L+G_R)^2}{H_L + H_R + \lambda} \right] - \gamma
$$

* Higher gain → better split.
* $\gamma$ acts like a **minimum split gain threshold**, avoiding overfitting.

---

### 3. Intuition Recap

1. Start with a weak model (tree).
2. Compute gradients ($g_i$) and Hessians ($h_i$) of loss wrt predictions.
3. Fit a tree to minimize these residuals.
4. Update predictions with optimal leaf weights $w_j^*$.
5. Repeat until convergence or stopping criteria.

---

**Why is XGBoost powerful?**

* Uses **2nd order optimization (Newton step)** for faster convergence.
* Has **built-in regularization** ($\lambda, \gamma$) → controls overfitting.
* Handles **sparse data** and **missing values** gracefully.
* Efficient (parallelism, cache awareness, pruning).




## When Do We Stop Splitting in XGBoost?

XGBoost uses several **stopping conditions** for splitting nodes:

---

### 1. **Maximum Depth (max\_depth)**

* Once a node reaches the maximum allowed depth, it cannot be split further.
* Example: `max_depth=3` → tree stops growing after 3 levels.
* Intuition: Prevents trees from growing too deep and memorizing training data.

---

### 2. **Minimum Child Weight (min\_child\_weight)**

* A split is stopped if the **sum of instance Hessians** (approx measure of data + confidence) in a leaf is smaller than this threshold.
* Ensures that a leaf has enough "data weight" to justify a split.
* Helps avoid creating leaves with very few samples.

---

### 3. **Minimum Loss Reduction (gamma)**

* XGBoost computes the **Gain** for each potential split:

$$
\text{Gain} = \frac{1}{2}\left[\frac{G_L^2}{H_L + \lambda} + \frac{G_R^2}{H_R + \lambda} - \frac{(G_L+G_R)^2}{H_L + H_R + \lambda}\right] - \gamma
$$

* If `Gain < gamma`, the split is **not performed**.
* Intuition: If splitting doesn’t improve the objective enough, stop.

---

### 4. **Maximum Leaves (max\_leaves)**

* If you use `max_leaves`, tree growth is stopped once the number of leaves hits this limit.
* Useful for controlling tree size directly (instead of depth).

---

### 5. **Sample Constraints**

* If a node has too few samples, splitting stops:

  * Controlled by parameters like `min_samples_split` (in sklearn API) or `min_child_weight`.

---

**Intuition Summary**

XGBoost stops splitting when **further splitting is not worth it**:

* Tree is too deep (`max_depth`).
* Too few samples in a node (`min_child_weight`).
* Split doesn’t improve loss enough (`gamma`).
* Reached max number of leaves (`max_leaves`).

This balances:

* **Overfitting** (tree grows too much → memorizes training set).
* **Underfitting** (tree stops too early → misses patterns).

---

⚖️ So, in practice:

* Use **`max_depth`** and **`gamma`** to control model complexity.
* Use **`min_child_weight`** to avoid noisy splits.
* Monitor **validation loss** and use **early\_stopping\_rounds** at the boosting level.
